<a class="reference external" 
    href="https://jupyter.designsafe-ci.org/hub/user-redirect/lab/tree/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Jupyter_Notebooks/tapis_appsDev_CustomApp_opsmps3copy.ipynb" 
    target="_blank"
    >
<img alt="Try on DesignSafe" src="https://raw.githubusercontent.com/DesignSafe-Training/pinn/main/DesignSafe-Badge.svg" /></a>

# GetWork App  📒
***Create your own Tapis App***
THIS WORKS, KIND OF, BUT IT'S SO CONVOLUTED AND STILL NOT CLEAR.
by Silvia Mazzoni, DesignSafe, 2025

Let's write an app to get your user and system-dependent path

## Workflow

| Step                        | Description                                                         |
| --------------------------- | ------------------------------------------------------------------- |
| 1. Create *app.json*        | Describes the app, its inputs, execution system, and wrapper script |
| 2. Create *tapisjob_app.sh* | Runs your analysis (e.g., ibrun OpenSees main.tcl)                  |
| 2a. Zip *tapisjob_app.sh* | It needs a zip file!                  |
| 3. Create *profile.json*    | (Optional) Loads modules/environment                                |
| 4. Upload Files             | To the deployment path in your storage system                       |
| 5. Register App             | With Tapis via CLI or Python                                        |
| 6. Submit Job               | Define `job.json` and submit                                        |

In [1]:
import json

In [2]:
# Local Utilities Library
# you can remove the logic associated with the local path
import sys,os
relativePath = '../OpsUtils'
if os.path.exists(relativePath):
    print("Using local utilities library")
    PathOpsUtils = os.path.expanduser(relativePath)
else:
    print('using communitydata')
    PathOpsUtils = os.path.expanduser('~/CommunityData/OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/OpsUtils')
if not PathOpsUtils in sys.path: sys.path.append(PathOpsUtils)
from OpsUtils import OpsUtils

Using local utilities library


---
## Connect to Tapis

In [3]:
t=OpsUtils.connect_tapis()

 -- Checking Tapis token --
 Token loaded from file. Token is still valid!
 Token expires at: 2025-08-20T18:35:53+00:00
 Token expires in: 0:56:45.401141
-- LOG IN SUCCESSFUL! --


---
### Get username

In [4]:
OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['get_tapis_username.py'])

In [5]:
username = OpsUtils.get_tapis_username(t)
print('username',username)

username silvia


## Define actions!

In [6]:
# Make these True once you are done validating
do_makeApp = True
do_submitJob = True

---
## Configure App

In [7]:
app_id = 'get-work-path-test'
app_description = 'get work path'

### AutoIncrement app_version

In [8]:
if do_makeApp:
    results = t.apps.getApps()
    foundIt = False
    for thisRes in results:
        here_id = thisRes.id
        if here_id == app_id:
            foundIt = True
    if foundIt:
        latest_app_version = OpsUtils.get_latest_app_version(t,app_id)
        print('app exists, now latest_app_version',latest_app_version)
        app_version = OpsUtils.bump_app_version(latest_app_version,'patch')
    else:
        print('new app')
        app_version = '0.0.1'
    print('now app_version',app_version)

app exists, now latest_app_version 0.0.6
now app_version 0.0.7


### Where to store the files

In [9]:
if do_makeApp:
    app_system_id="designsafe.storage.default"
    app_path = f"{username}/apps/{app_id}/{app_version}"
    container_filename = f'{app_id}.zip'

### Define folder structure: app_id/version

In [10]:
if do_makeApp:
    app_folderName = f'~/MyData/myAuthoredTapisApps/{app_id}/{app_version}'; # your choice

In [11]:
if do_makeApp:
    app_folder = os.path.abspath(os.path.expanduser(app_folderName))
    os.makedirs(app_folder, exist_ok=True)
    print(f'app_folder: {app_folder}\n exists:',os.path.exists(app_folder))

app_folder: /home/jupyter/MyData/myAuthoredTapisApps/get-work-path-test/0.0.7
 exists: True


---
## Create the App Files

A Tapis app needs the following **core files**:

* **Readme.MD** – App Description (OPTIONAL)<br>
This file is helpful in communicating content to the app user.  

* **app.json** – App Definition<br>
Defines the app’s metadata, inputs, parameters, and execution configuration.    

* **profile.json** – Environment Setup *(optional but common)*<br>
Loads modules or sets environment variables on the execution system. This file will load modules before your script runs. This is executed on the compute node.

* **tapisjob_app.sh** – Wrapper Script<br>
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [12]:
# let's collect the list of files we are creating as we will need this list in upload
if do_makeApp:
    appFilesList = [];

#### **Readme.MD** – App Description
This file is helpful in communicating content to the app user.  

In [13]:
if do_makeApp:
    thisFilename = 'ReadMe.MD'
    thisText = """\
    # making a dummy app to get user-and-system-specific work path
    """
    
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **app.json** – App Definition
Defines the app’s metadata, inputs, parameters, and execution configuration.    

In [14]:
if do_makeApp:
    thisFilename = 'app.json'
    thisText = """\
    {
      "id": "__app_id__",
      "name": "__app_id__",
      "version": "__app_version__",
      "description": "__app_description__",
      "owner": "${apiUserId}",
    
      "executionSystem": "designsafe.community.execution",
      "deploymentSystem": "designsafe.storage.default",
      "deploymentPath": "apps/bash-echo/0.0.2",
      "templatePath": "wrapper.sh",
    
      "executionType": "HPC",
      "runtime": "SINGULARITY",
      "containerImage": "docker://rockylinux:9",
    
      "jobType": "BATCH",
      "parallelism": "SERIAL",
      "defaultNodes": 1,
      "defaultProcessors": 1,
      "defaultMaxRunTime": "00:05:00",
      "defaultMemory": "1GB",
    
      "tags": ["utility", "env"],
      "inputs": [],
      "parameters": [
        {
          "id": "SCRIPT",
          "value": { "required": false, "default": "printenv.sh", "type": "string", "visible": true }
        }
      ],
      "archive": true,
      "archiveOnAppError": true
    }

    """
    thisText = thisText.replace("__app_id__", app_id)
    thisText = thisText.replace("__app_version__", app_version)
    thisText = thisText.replace("__app_description__", app_description)
    thisText = thisText.replace("__container_filename_path__", f"tapis://{app_system_id}/{app_path}/{container_filename}")
    thisText = thisText.replace("__container_filename__", container_filename)
  
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **profile.json** – Environment Setup
This file will load modules before your script runs. This is executed on the compute node.

In [15]:
if do_makeApp:
    thisFilename = 'profile.json'
    thisText = """\
    {
      "modules": []
    }
    """ 
    with open(f"{app_folder}/{thisFilename}", "w") as f:
        f.write(thisText)

    appFilesList.append(thisFilename)

#### **tapisjob_app.sh** – Wrapper Script
Wrapper script executed by the job; this is the command that launches your code (e.g., runs *OpenSeesMP*, Python, or a script)

In [16]:
if do_makeApp:
    import textwrap, time
    from zipfile import ZipFile, ZIP_DEFLATED, ZipInfo
    
    thisFilename_sh = "tapisjob_app.sh"
    thisFilename = container_filename  # existing var


    bash_script = textwrap.dedent("""\
        #!/bin/bash
        set -euo pipefail
        set -x

        inputDirectory="${inputDirectory:?inputDirectory not set}"
        echo "$inputDirectory"

        # Run the user-provided script path (passed via appArg SCRIPT)
        SCRIPT="${1:-./printenv.sh}"
        bash "$SCRIPT"
        
    """)


    zip_path = os.path.join(app_folder, thisFilename)
    print('zip_path',zip_path)

    # write with executable permission inside the ZIP
    zi = ZipInfo(thisFilename_sh)
    zi.date_time = time.localtime(time.time())[:6]
    zi.compress_type = ZIP_DEFLATED
    zi.external_attr = 0o100755 << 16   # -rwxr-xr-x on a regular file
    
    with ZipFile(zip_path, "w", ZIP_DEFLATED) as z:
        z.writestr(zi, bash_script)
        
    appFilesList.append(thisFilename)

zip_path /home/jupyter/MyData/myAuthoredTapisApps/get-work-path-test/0.0.7/get-work-path-test.zip


#### File Check
Look at the files we have written and check for typos or formatting errors.

In [17]:
if do_makeApp:
    print(appFilesList)
    OpsUtils.show_text_file_in_accordion(app_folder, appFilesList)

['ReadMe.MD', 'app.json', 'profile.json', 'get-work-path-test.zip']


---
## Validate App Locally

In [18]:
if do_makeApp:
    OpsUtils.show_text_file_in_accordion(PathOpsUtils, ['validate_app_folder.py'])

In [19]:
if do_makeApp:
    validation = OpsUtils.validate_app_folder(app_folder,appFilesList)
    if not validation:
        print('Validation Failed: stopping here!!!!')
        a = 3/0

🔍 Validating app folder: /home/jupyter/MyData/myAuthoredTapisApps/get-work-path-test/0.0.7

✅ All required files are present.

📄 App ID: get-work-path-test
📄 App Name: get-work-path-test
📄 Version: 0.0.7
🔧 Parameters: ['SCRIPT']
📦 Inputs: []
📤 Outputs: []

App Keys: ['id', 'name', 'version', 'description', 'owner', 'executionSystem', 'deploymentSystem', 'deploymentPath', 'templatePath', 'executionType', 'runtime', 'containerImage', 'jobType', 'parallelism', 'defaultNodes', 'defaultProcessors', 'defaultMaxRunTime', 'defaultMemory', 'tags', 'inputs', 'parameters', 'archive', 'archiveOnAppError', 'modules']

✅ Basic validation complete. App folder looks good!


---
## Deploy the App

### Make the directory inside your 'MyData' on designsafe.storage.default.
The apps in this folder are the ones that area actually uploaded.

In [20]:
if do_makeApp:
    t.files.mkdir(systemId=app_system_id, path=app_path)
    print('app_path',app_path)

app_path silvia/apps/get-work-path-test/0.0.7


### Upload files to your deployment system (e.g., DesignSafe default storage)
Using Tapipy (Python SDK) in a Jupyter Notebook

In [21]:
if do_makeApp:
    for fname in appFilesList:
        fpath = f'{app_folder}/{fname}'
        t.upload(source_file_path=fpath,
                 system_id=app_system_id,
                 dest_file_path=f'{app_path}/{fname}')

#### Verify upload

In [22]:
if do_makeApp:
    print('app_system_id:',app_system_id)
    print('app_path:',app_path)
    appfiles = t.files.listFiles(systemId=app_system_id, path=app_path)
    for thisF in appfiles:
        print(thisF)
        print('')

app_system_id: designsafe.storage.default
app_path: silvia/apps/get-work-path-test/0.0.7

group: 819066
lastModified: 2025-08-20T17:39:09Z
mimeType: application/json
name: app.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/get-work-path-test/0.0.7/app.json
size: 965
type: file
url: tapis://designsafe.storage.default/silvia/apps/get-work-path-test/0.0.7/app.json


group: 819066
lastModified: 2025-08-20T17:39:09Z
mimeType: application/zip
name: get-work-path-test.zip
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/get-work-path-test/0.0.7/get-work-path-test.zip
size: 288
type: file
url: tapis://designsafe.storage.default/silvia/apps/get-work-path-test/0.0.7/get-work-path-test.zip


group: 819066
lastModified: 2025-08-20T17:39:09Z
mimeType: application/json
name: profile.json
nativePermissions: rw-rw----
owner: 843714
path: silvia/apps/get-work-path-test/0.0.7/profile.json
size: 36
type: file
url: tapis://designsafe.storage.default/silvia/apps/get-work-path

---
## Register the App
This creates the actual App record that Jobs can run.

Do this Using Tapipy (Python)

In [23]:
if do_makeApp:
    # Create (or create a new version) of the app
    with open(f'{app_folder}/app.json') as f:
        app_def = json.load(f)
    t.apps.createAppVersion(**app_def)

### Check that app is up

#### List all apps

In [24]:
listType = 'ALL' # Include all items requester is authorized to view. Includes check for READ or MODIFY permission.
select = 'id,created,description,version,owner' # Attributes to return in each result.
orderBy = 'created(asc)'
results = t.apps.getApps( orderBy=orderBy,
                         select=select)  
for thisRes in results:
    print('--')
    print(thisRes)

--

created: 2024-12-13T20:15:33.014747Z
description: Run shell commands on remote systems
id: shell-runner-1.0.0
owner: silvia
version: 1.0.0
--

created: 2024-12-14T15:49:21.510206Z
description: Run shell commands on remote systems
id: shell-runner
owner: silvia
version: 1.0.0
--

created: 2025-08-16T16:51:35.355715Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia
owner: silvia
version: latest
--

created: 2025-08-16T18:53:58.422294Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-silvia-new
owner: silvia
version: latest
--

created: 2025-08-17T22:34:01.193589Z
description: Runs all the processors in parallel. Requires understanding of parallel processing and the capabilities to write parallel scripts.
id: opensees-mp-s3-copy-silvia
owner: silvia
vers

#### List the new app

In [25]:
appMetaData = t.apps.getAppLatestVersion(appId=app_id)
print(appMetaData)


containerImage: docker://rockylinux:9
created: 2025-08-20T17:39:09.191106Z
deleted: False
description: get work path
enabled: True
id: get-work-path-test
isPublic: False
jobAttributes: 
archiveOnAppError: False
archiveSystemDir: None
archiveSystemId: None
cmdPrefix: None
coresPerNode: 1
description: None
dtnSystemInputDir: !tapis_not_set
dtnSystemOutputDir: !tapis_not_set
dynamicExecSystem: False
execSystemConstraints: None
execSystemExecDir: None
execSystemId: None
execSystemInputDir: None
execSystemLogicalQueue: None
execSystemOutputDir: None
fileInputArrays: []
fileInputs: []
isMpi: False
maxMinutes: 10
memoryMB: 100
mpiCmd: None
nodeCount: 1
parameterSet: 
appArgs: []
archiveFilter: 
excludes: []
includeLaunchFiles: True
includes: []
containerArgs: []
envVariables: []
logConfig: 
stderrFilename: 
stdoutFilename: 
schedulerOptions: []
subscriptions: []
tags: []
jobType: BATCH
locked: False
maxJobs: 2147483647
maxJobsPerUser: 2147483647
notes: 

owner: silvia
runtime: SINGULARITY
ru

---
## Submit a Job

You can now submit a job using this app. You can use the Tapis CLI, Tapipy, or a web form.

We are using TapiPy directly from this notebook. We will not specify a version so that the latest is used by default in the description.

---
### Initialize

In [26]:
tapisInputAll = {}

---
### SLURM-Specific Input

In [27]:
tapisInputAll["maxMinutes"] = 1

tapisInputAll["execSystemId"] = "stampede3"
tapisInputAll["execSystemLogicalQueue"] = "skx-dev"
tapisInputAll["nodeCount"] = 1
tapisInputAll["coresPerNode"] = 48
tapisInputAll["allocation"] = "DS-HPC1"

tapisInputAll['archive_system']='Work' # Options: MyData or Work

---
### App-Specific Input

In [28]:
print('app_id:',app_id)

app_id: get-work-path-test


In [29]:


# tapisInput['storage_system'] = 'CommunityData'
# tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 


# # -----------------------------------------------------
# if do_submitJob:
#     here_tapisInput = tapisInput.copy()
#     here_tapisInput["appId"] = app_id 
#     here_tapisInput["name"] = here_tapisInput["appId"]
#     print(f'\n -- {here_tapisInput['name']} --\n')
#     jobReturns = OpsUtils.run_tapis_job(t,here_tapisInput,get_job_metadata=True,get_job_history=True,get_job_filedata=True,askConfirmJob = False,askConfirmMonitorRT = False)
# # -----------------------------------------------------

In [30]:
   
# from tapipy.tapis import TapisResult
# appMetaData = t.apps.getAppLatestVersion(appId=app_id)
# app_MetaData = appMetaData.__dict__
# print(app_MetaData.keys())
# app_jobAttributes = app_MetaData['jobAttributes'].__dict__
# print('app_jobAttributes',app_jobAttributes.keys())
# app_parameterSet = app_jobAttributes['parameterSet'].__dict__
# print('app_parameterSet',app_parameterSet.keys())
# # app_appArgs = app_parameterSet['appArgs']
# # app_envVariables = app_parameterSet['envVariables']
# for app_key in ['appArgs','envVariables']:
#     print('app_key',app_key)
#     app_Dict = app_parameterSet[app_key]
#     print(app_Dict)
# for app_key in ['fileInputs']:
#     print('app_key',app_key)
#     app_Dict = app_jobAttributes[app_key]
#     print(app_Dict)
    

# job_description = {'name':'getWork'}
# job_description["appId"] = app_id
# job_description["appVersion"] = app_version
# here_tapisInput = tapisInputAll.copy()
# here_tapisInput['storage_system'] = 'CommunityData'
# here_tapisInput['input_folder'] = 'OpenSees/TrainingMaterial/training-OpenSees-on-DesignSafe/Examples_OpenSees/BasicExamples' 


In [33]:
job_def = {
    "name": "print-work",
    "appId": "bash-echo",
    "appVersion": "0.0.2",
    "archiveSystemId": "designsafe.storage.default",
    "archiveSystemDir": f"/{t.username}/tapis/jobs/print-work-${{JobUUID}}",
    "fileInputs": [
        {
            "sourceUrl": f"tapis://designsafe.storage.default/{t.username}/tapis/printenv/printenv.sh",
            "targetPath": "."
        }
    ],
    "parameterSet": {
        "appArgs": [ { "name": "SCRIPT", "value": "./printenv.sh" } ],
        "envVariables": []
    }
}
tapisInput = {}

In [35]:
OpsUtils.run_tapis_job(t,tapisInput,job_description = job_def)

Are you sure you want to submit the job? (press n to cancel, any key to confirm):  


Submitting Job


BadRequestError: message: NET_REQUEST_PAYLOAD_ERROR Error accessing payload for submitJob request: TAPIS_JSON_VALIDATION_ERROR JSON validation error: TAPIS_JSON_VALIDATION_FAILURE JSON validation error: #/parameterSet/appArgs/0: 2 schema violations found  #1#/parameterSet/appArgs/0: required key [arg] not found #2#/parameterSet/appArgs/0: extraneous key [value] is not permitted